In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt

In [2]:
from annchor.distances import wasserstein
from annchor import compare_neighbor_graphs



In [3]:
np.random.seed(1)
nx = 10000
X=np.random.random(size=(nx,5))
from numba import njit
@njit
def f(x,y):
    return np.linalg.norm(x-y)

f(X[nx-3],X[3])

0.8606460035963592

In [4]:
from annchor import BruteForce

start_time = time.time()

bruteforce = BruteForce(X,f)
bruteforce.get_neighbor_graph()

print('Brute Force Time: %5.3f seconds' % (time.time()-start_time))

neighbor_graph = bruteforce.neighbor_graph




Brute Force Time: 92.293 seconds


In [17]:
from annchor import Annchor
k=25
start_time = time.time()

# Call ANNchor
ann = Annchor(X,
              f,
              n_anchors=30,
              n_neighbors=k,
              n_samples=5000,
              p_work=0.02,
             verbose=True)

ann.fit()

print('ANNchor Time: %5.3f seconds' % (time.time()-start_time))

# Test accuracy
error = compare_neighbor_graphs(neighbor_graph,
                                ann.neighbor_graph,
                                k)
print('ANNchor Accuracy: %d incorrect NN pairs (%5.3f%%)' % (error,100*error/(k*nx)))

computing anchors...
get_anchors: 0.8531908988952637
computing locality...
get_locality: 2.3705790042877197
computing features...
get_features: 23.958975076675415
computing sample...
get_sample: 35.947726011276245
fitting regression...
fit_predict_regression: 47.513323068618774
fitting errors...
fit_predict_errors: 48.84856700897217
selecting/refining candidate pairs (1)
select_refine_candidate_pairs: 61.950567960739136
selecting/refining candidate pairs (2)
select_refine_candidate_pairs: 74.28281307220459
generating neighbour graph
get_ann: 75.58573913574219
ANNchor Time: 75.605 seconds
ANNchor Accuracy: 7 incorrect NN pairs (0.003%)


In [277]:
start = time.time()
IJs = np.hstack([ fF(self.check,i) for i in range(self.nx)])
IJs = gG(IJs,self.nx)
IJs = IJs.T


n = IJs.shape[0]

isort = np.arange(n).astype(np.int64)
jsort = np.argsort( IJs[:,1]).astype(np.int64)
fi = IJs[:,0]
fj = IJs[jsort,1]

In [283]:
np.all(IJs[:,0]==np.sort(IJs[:,0]))

True